# HADOOP ECOSYSTEM CONTINUED

First, let's start the daemons again:

First ssh:

In [ ]:
sudo service ssh stop
sudo service ssh start

In [ ]:
service ssh status

And then hdfs:

In [ ]:
start-dfs.sh 2> /dev/null

And yarn:

In [ ]:
start-yarn.sh 2> /dev/null

And the job history server (so that PIG does not complain that the server does not respond):

In [ ]:
mr-jobhistory-daemon.sh --config $HADOOP_CONF_DIR start historyserver 2> /dev/null

Check whether hdfs and yarn daemons are running:

In [ ]:
jps

Check the status of hdfs:

In [ ]:
hdfs dfsadmin -report 2> /dev/null

And if hdfs is in safemode, stop safemode:

In [ ]:
hdfs dfsadmin -safemode leave 2> /dev/null

And start postgresql:

In [ ]:
sudo service postgresql start

In [ ]:
psql -U postgres -d imdb2 -c "\dt+"

## HIVE

Hive is a data warehouse system and MapReduce wrapper that presents an SQL interface. So SQL programmers feel at home through Hive in the Hadoop Ecosystem, without writing MapReduce jobs explicitly

First start the metastore, the central repository to store metadata for hive tables:

You should rerun this if the hive or pig commands return an error saying that "thrift server does not respond"

In [ ]:
hive --service metastore &

And check whether the service is listening on its port:

In [ ]:
netstat -an | grep 9083

### IMPORT DATA WITH SQOOP

Sqoop can import the data as Hive tables with "--hive-import --create-hive-table" flags

First flush the directory for default import location:

In [ ]:
hdfs dfs -rm -r /user/jovyan/title_ratings/ 2> /dev/null

In [ ]:
sqoop import \
--connect jdbc:postgresql://localhost:5432/imdb2 \
--username postgres \
--table title_ratings \
--hive-import --create-hive-table --direct \
2> /dev/null

And view the imported file:

In [ ]:
hdfs dfs -find / -name "title*ratings" 2> /dev/null

In [ ]:
hdfs dfs -ls /user/hive/warehouse/ 2> /dev/null

And let's check from hive whether files are imported:

In [ ]:
hive -e 'show tables' 2> /dev/null

Let's try to read the file:

In [ ]:
hdfs dfs -cat /user/hive/warehouse/title_ratings/* 2> /dev/null | head

If we want to delete the table, we do it from the hive command, not by manually deleting the file:

In [ ]:
hive -e 'drop table title_ratings' 2> /dev/null

And cheeck again whether the table is delted:

In [ ]:
hive -e 'show tables' 2> /dev/null

In [ ]:
hdfs dfs -ls /user/hive/warehouse/ 2> /dev/null

Let's import title_ratings again for further commands:

In [ ]:
hdfs dfs -rm -r /user/jovyan/title_ratings/ 2> /dev/null

In [ ]:
sqoop import \
--connect jdbc:postgresql://localhost:5432/imdb2 \
--username postgres \
--table title_ratings \
--hive-import --create-hive-table --direct \
2> /dev/null

And check again:

In [ ]:
hive -e 'show tables' 2> /dev/null

### A SQOOP TO HIVE IMPORT EXERCISE

Now you'll import a postgresql database into hdfs as text files and hive tables

Now you'll 
- first import a publicly available sql dump into postgresql
- Then import the postgresql database into hdfs as text files and hive tables

The database is "World", containing list of cities, countries and languages. The link is

http://pgfoundry.org/frs/download.php/527/world-1.0.tar.gz

There are other sample databases following the links:

http://pgfoundry.org/projects/dbsamples

https://wiki.postgresql.org/wiki/Sample_Databases

First download the file

In [ ]:
wget http://pgfoundry.org/frs/download.php/527/world-1.0.tar.gz

And extract the archive:

In [ ]:
tar -xzvf world-1.0.tar.gz

Create a new database at postgresql:

In [ ]:
createdb -U postgres world2

And import the dump into the database:

In [ ]:
psql -U postgres world2 < dbsamples-0.1/world/world.sql

View the tables:

In [ ]:
psql -U postgres -d world2 -c "\dt"

If you've missed the steps, no worry. There is already a copy of the database named **world** in postgresql. We can query it

View the tables:

In [ ]:
psql -U postgres -d world -c "\dt"

And view the fields of the tables:

In [ ]:
psql -U postgres -d world -c "\d+ public.*"

And view some of the rows of the tables:

In [ ]:
psql -U postgres -d world -c "select * from city limit 10"

In [ ]:
psql -U postgres -d world -c "select * from country limit 10"

In [ ]:
psql -U postgres -d world -c "select * from countrylanguage limit 10"

We may also view its entity relationship diagram:

![aa](world_erd.png)

And let's import into hive:

First get the list of tables:

In [ ]:
tables=$(psql -U postgres -d world -t --pset="border=0" -c "\dt" | \
awk -F " " '{ print $2 }')

In [ ]:
echo "$tables"

Create a hive database to import into:

In [ ]:
hive -e "create database world" 2> /dev/null

And import each table in the postgresql database into hive database:

In [ ]:
echo "$tables" | while read l;
do
    sqoop import \
    --connect jdbc:postgresql://localhost:5432/world \
    --username postgres \
    --table $l \
    --hive-import \
    --create-hive-table \
    --hive-table world.$l \
    --direct \
    2> /dev/null
done

### HIVE OPERATIONS

First show tables:

In [ ]:
hive -e "show tables" 2> /dev/null

Show tables in world database:

In [ ]:
hive -e "show tables in world" 2> /dev/null

Then let's create a database from hive:

In [ ]:
hive -e "create database deneme" 2> /dev/null

List the databases:

In [ ]:
hive -e "show databases" 2> /dev/null

Delete deneme database:

In [ ]:
hive -e "drop database deneme" 2> /dev/null

And let's run a simple query inside a hive database:

In [ ]:
hive -e "select * from title_ratings limit 1;"  2> /dev/null

Now that we have a hive database on hdfs, similar to our postgresql database, we can run similar queries

In fact, under the hood, Hive converts the HiveQL query to a series of map reduce jobs

The plan of the conversion can be viewed by prefixing the statement with "explain"

In [ ]:
hive -e "explain
select * from title_ratings limit 1;" \
2> /dev/null

Hive is suitable for simpler queries. However as the queries get more complex and need a clearer definition of the dataflow, we should revert to a tool such as Pig

**Exercise 1:**

Please:
- Create an imdb database in hive
- Import the first 10.000 rows of title_episode table from imdb2 database on postgresql into imdb database in hive
- Check the the table is imported

Note that:
- You should use the **--query** argument with the sqoop (check Session 6 notebook)
- When you use **--query**, you cannot use **--table** argument at the same time
- When you use **--query**, you also have to specify a temporary import directory with the **--target-dir** option (it should be a non-existent directory)
- You should add "WHERE \\\\$CONDITIONS" to your query in sqoop

In [ ]:
pass1=
solution=$(cat ~/crypt/sqoop_to_hive.crypt | openssl enc -md sha256 -aes-128-cbc -a -d -salt -pass pass:$pass1 2> /dev/null)
echo "$solution"; echo
for l in "$solution"; do eval "${l}"; done

### A HIVE EXERCISE WITH WORLD DATABASE

See the tables in the world database:

In [ ]:
hive -e "show tables in world" 2> /dev/null

In order get information about tables, you can run:

In [ ]:
hive -e "show create table world.city;" 2> /dev/null

The output will return information on the schema (column names and types), file type and size information

You can repeat it for other tables in the database

Now let's run a very simple query:

In [ ]:
hive -e "use world;
select * from country limit 1;" 2> /dev/null

Now let's run a query to see the average lifeexpantancy of all countries:

In [ ]:
hive -e "use world;
select avg(lifeexpectancy) from country limit 10;" 2> /dev/null

Now the next task is:

- get the names (from country table) of the countries, official languages (from country languages) of which include english 

You should get:

```mysql
OK
American Samoa
Anguilla
Antigua and Barbuda
Australia
Barbados
Belize
Bermuda
United Kingdom
Virgin Islands, British
Cayman Islands
South Africa
Falkland Islands
Gibraltar
Guam
Hong Kong
Ireland
Christmas Island
Canada
Cocos (Keeling) Islands
Lesotho
Malta
Marshall Islands
Montserrat
Nauru
Niue
Norfolk Island
Palau
Northern Mariana Islands
Saint Helena
Saint Kitts and Nevis
Saint Lucia
Saint Vincent and the Grenadines
Samoa
Seychelles
Tokelau
Tonga
Turks and Caicos Islands
Tuvalu
New Zealand
Vanuatu
United States
Virgin Islands, U.S.
Zimbabwe
United States Minor Outlying Islands
Time taken: 11.424 seconds, Fetched: 44 row(s)
```

In [ ]:
hive -e "use world;
select c.name from country c left join
countrylanguage l on c.code=l.countrycode
where l.isofficial = true
and l.language = 'English';"  2> /dev/null

**Exercise 2:**

Query world database in hive such that:
- The query should report names of countries and its capital cities and also the proportion of the population of the capital to the country as capshare
- country table should be left joined to city table. Check the ER diagram above again
- filter for countries with population above 0
- order by descending capshare

You can first try your query on postgresql server's world database

The result should be:

```Bash
Singapore                              Singapore                          112.63619287917017
Gibraltar                              Gibraltar                          108.1
Macao                                  Macao                              92.49471458773785
Pitcairn                               Adamstown                          84.0
Saint Pierre and Miquelon              Saint-Pierre                       82.97142857142858
Falkland Islands                       Stanley                            81.8
Palau                                  Koror                              63.1578947368421
Djibouti                               Djibouti                           60.03134796238245
Cook Islands                           Avarua                             59.5
Qatar                                  Doha                               59.26544240400668
Western Sahara                         El-Aaiún                           57.67918088737201
Bahamas                                Nassau                             56.02605863192182
Seychelles                             Victoria                           53.246753246753244
Cayman Islands                         George Town                        51.578947368421055
Holy See (Vatican City State)          Città del Vaticano                 45.5
Svalbard and Jan Mayen                 Longyearbyen                       44.9375
Marshall Islands                       Dalap-Uliga-Darrit                 43.75
Norfolk Island                         Kingston                           40.0
Iceland                                Reykjavík                          39.13405017921147
Tuvalu                                 Funafuti                           38.333333333333336
Virgin Islands, British                Road Town                          38.095238095238095
Uruguay                                Montevideo                         37.03925681750075
New Caledonia                          Nouméa                             35.65093457943925
Armenia                                Yerevan                            35.47443181818182
Antigua and Barbuda                    Saint John´s                       35.294117647058826
Gabon                                  Libreville                         34.176182707993476
Niue                                   Alofi                              34.1
Faroe Islands                          Tórshavn                           33.818604651162794
Sao Tome and Principe                  São Tomé                           33.701360544217685
Lebanon                                Beirut                             33.516148689823275
Congo                                  Brazzaville                        32.27998640842677
Latvia                                 Riga                               31.529081758930783
Chile                                  Santiago de Chile                  30.92468608244034
Saint Kitts and Nevis                  Basseterre                         30.526315789473685
Libyan Arab Jamahiriya                 Tripoli                            30.008920606601247
Guyana                                 Georgetown                         29.500580720092916
Mongolia                               Ulan Bator                         29.064613072877535
Turks and Caicos Islands               Cockburn Town                      28.235294117647058
Aruba                                  Oranjestad                         28.188349514563107
Estonia                                Tallinn                            28.069830461367427
French Guiana                          Cayenne                            28.010497237569062
Christmas Island                       Flying Fish Cove                   28.0
Cocos (Keeling) Islands                West Island                        27.833333333333332
Andorra                                Andorra la Vella                   27.165384615384614
Liberia                                Monrovia                           26.94990488268865
Suriname                               Paramaribo                         26.85851318944844
Cyprus                                 Nicosia                            25.83808135683053
Peru                                   Lima                               25.19169589275972
Saint Helena                           Jamestown                          25.0
Mauritania                             Nouakchott                         24.99250936329588
Georgia                                Tbilisi                            24.863123993558776
Maldives                               Male                               24.825174825174827
Greenland                              Nuuk                               24.008928571428573
Bahrain                                al-Manama                          23.987034035656404
Martinique                             Fort-de-France                     23.810126582278482
Azerbaijan                             Baku                               23.116110680113785
Dominica                               Roseau                             22.877464788732393
Tonga                                  Nuku´alofa                         22.626262626262626
Cape Verde                             Praia                              22.149532710280372
Macedonia                              Skopje                             21.95153162055336
South Korea                            Seoul                              21.308212364443687
Cuba                                   La Habana                          20.141058834032677
Samoa                                  Apia                               19.944444444444443
Austria                                Wien                               19.87374873328555
Guinea-Bissau                          Bissau                             19.868095630667767
Jordan                                 Amman                              19.673421207948063
Hong Kong                              Victoria                           19.354718372161603
Dominican Republic                     Santo Domingo de Guzmán            18.951924661565627
Nicaragua                              Managua                            18.900275916436737
Réunion                                Saint-Denis                        18.80972818311874
Iraq                                   Baghdad                            18.758382003028338
Luxembourg                             Luxembourg [Luxemburg/Lëtzebuerg]  18.521918751434473
Montserrat                             Plymouth                           18.181818181818183
Hungary                                Budapest                           18.03759757846105
Vanuatu                                Port-Vila                          17.736842105263158
Sierra Leone                           Freetown                           17.511330861145446
Panama                                 Ciudad de Panamá                   16.504656862745097
Moldova                                Chisinau                           16.43607305936073
Belarus                                Minsk                              16.354044548651817
United Arab Emirates                   Abu Dhabi                          16.3332650553052
Croatia                                Zagreb                             15.800804828973844
Angola                                 Luanda                             15.701195837863022
Lithuania                              Vilnius                            15.627119102338787
Liechtenstein                          Vaduz                              15.613003095975232
Saudi Arabia                           Riyadh                             15.383903364650346
Tokelau                                Fakaofo                            15.0
Saint Vincent and the Grenadines       Kingstown                          15.0
Colombia                               Santafé de Bogotá                  14.793747784787694
Guinea                                 Conakry                            14.678465679676986
Central African Republic               Bangui                             14.495159059474412
Zambia                                 Lusaka                             14.36361653397317
Virgin Islands, U.S.                   Charlotte Amalie                   13.978494623655914
Lesotho                                Maseru                             13.794705062703205
Bulgaria                               Sofija                             13.701815429317902
Slovenia                               Ljubljana                          13.632457993761948
Botswana                               Gaborone                           13.132983970406904
Ireland                                Dublin                             12.764006251490027
Honduras                               Tegucigalpa                        12.550501156515034
Kyrgyzstan                             Bishkek                            12.54309427537774
Ecuador                                Quito                              12.44233749802309
United Kingdom                         London                             12.218357222164451
Turkmenistan                           Ashgabat                           12.123794572774164
Zimbabwe                               Harare                             12.083297626189047
Mauritius                              Port-Louis                         11.934369602763384
Taiwan                                 Taipei                             11.867864845434939
Northern Mariana Islands               Garapan                            11.794871794871796
Czech Republic                         Praha                              11.491676477169905
Palestine                              Gaza                               11.403805224121252
Norway                                 Oslo                               11.359294406609356
Yugoslavia                             Beograd                            11.31578947368421
Solomon Islands                        Honiara                            11.283783783783784
Puerto Rico                            San Juan                           11.22703540966658
Eritrea                                Asmara                             11.194805194805195
French Polynesia                       Papeete                            10.873617021276596
Haiti                                  Port-au-Prince                     10.757382631963026
Finland                                Helsinki [Helsingfors]             10.741476998046913
North Korea                            Pyongyang                          10.33320853612879
Thailand                               Bangkok                            10.293610645124513
Israel                                 Jerusalem                          10.193019141064822
Paraguay                               Asunción                           10.148762736535662
Iran                                   Teheran                            9.98322796963162
Egypt                                  Cairo                              9.91599094493939
Somalia                                Mogadishu                          9.87422006536595
Congo, The Democratic Republic of the  Kinshasa                           9.80369380880474
Namibia                                Windhoek                           9.791425260718425
Laos                                   Vientiane                          9.788330572427757
Fiji Islands                           Suva                               9.46952264381885
Denmark                                København                          9.300168855534709
Bolivia                                La Paz                             9.10242526113579
Cameroon                               Yaoundé                            9.100430891614186
Bosnia and Herzegovina                 Sarajevo                           9.06344410876133
Romania                                Bucuresti                          8.97833938233395
Equatorial Guinea                      Malabo                             8.830022075055188
Uzbekistan                             Toskent                            8.707541738629821
Mexico                                 Ciudad de México                   8.688533692013632
San Marino                             San Marino                         8.496296296296297
Tajikistan                             Dushanbe                           8.468002585649645
Sweden                                 Stockholm                          8.467601056266505
Costa Rica                             San José                           8.429803629132488
Syria                                  Damascus                           8.353488372093024
Slovakia                               Bratislava                         8.303702743253005
Senegal                                Dakar                              8.280466195548993
Venezuela                              Caracas                            8.172503103020274
Togo                                   Lomé                               8.101101749837978
Mayotte                                Mamoutzou                          8.053691275167786
Argentina                              Buenos Aires                       8.052889392957443
Albania                                Tirana                             7.9383746912854285
Afghanistan                            Kabul                              7.834507042253521
Kenya                                  Nairobi                            7.613031914893617
Wallis and Futuna                      Mata-Utu                           7.58
Anguilla                               The Valley                         7.4375
Myanmar                                Rangoon (Yangon)                   7.370371182390213
Greece                                 Athenai                            7.321202006505021
Spain                                  Madrid                             7.29951295202793
Guatemala                              Ciudad de Guatemala                7.231453667105841
Micronesia, Federated States of        Palikir                            7.226890756302521
Mali                                   Bamako                             7.206266690404131
Tunisia                                Tunis                              7.204256206968496
Chad                                   N´Djaména                          6.939811789308587
Burkina Faso                           Ouagadougou                        6.90290692803887
Algeria                                Alger                              6.888881827714404
El Salvador                            San Salvador                       6.6180050987890375
Brunei                                 Bandar Seri Begawan                6.55
Japan                                  Tokyo                              6.297828179995896
Comoros                                Moroni                             6.228373702422146
Swaziland                              Mbabane                            6.051587301587301
Malaysia                               Kuala Lumpur                       5.8331505124977525
Russian Federation                     Moscow                             5.709502225488995
Portugal                               Lisboa                             5.633452028486837
East Timor                             Dili                               5.4124293785310735
Ghana                                  Accra                              5.293884820898477
Ukraine                                Kyiv                               5.200570794355478
Mozambique                             Maputo                             5.177530487804878
Papua New Guinea                       Port Moresby                       5.138339920948616
Cambodia                               Phnom Penh                         5.1052560888252145
Grenada                                Saint George´s                     4.915957446808511
Nauru                                  Yaren                              4.658333333333333
Netherlands                            Amsterdam                          4.609178013111447
Italy                                  Roma                               4.583184812760056
Turkey                                 Ankara                             4.562416843116938
Indonesia                              Jakarta                            4.528327683669091
Burundi                                Bujumbura                          4.480955937266617
New Zealand                            Wellington                         4.316416364577939
Madagascar                             Antananarivo                       4.238295069627399
Poland                                 Warszawa                           4.179090692716849
Germany                                Berlin                             4.121802915363897
Uganda                                 Kampala                            4.0903664248324
Jamaica                                Kingston                           4.024854819976771
Malawi                                 Lilongwe                           3.990517162471396
Ethiopia                               Addis Abeba                        3.987852633261408
Niger                                  Niamey                             3.91425908667288
Rwanda                                 Kigali                             3.6984352773826457
Monaco                                 Monaco-Ville                       3.6294117647058823
France                                 Paris                              3.588384772151279
Sri Lanka                              Colombo                            3.4259308440006375
American Samoa                         Fagatogo                           3.416176470588235
Trinidad and Tobago                    Port-of-Spain                      3.351042471042471
Gambia                                 Banjul                             3.2433716475095786
Sudan                                  Khartum                            3.2128958969142083
Benin                                  Porto-Novo                         3.18189273413154
Belize                                 Belmopan                           2.9481327800829877
Bangladesh                             Dhaka                              2.7972978204482986
Yemen                                  Sanaa                              2.7804770318021204
Guadeloupe                             Basse-Terre                        2.726535087719298
Kiribati                               Bairiki                            2.6819277108433734
Nepal                                  Kathmandu                          2.473192645215211
Barbados                               Bridgetown                         2.248148148148148
Morocco                                Rabat                              2.1990652887023385
Philippines                            Manila                             2.0812747640422815
Oman                                   Masqat                             2.04441384736428
Kazakstan                              Astana                             1.9182641928126734
Malta                                  Valletta                           1.8603366649132036
Bermuda                                Hamilton                           1.8461538461538463
Vietnam                                Hanoi                              1.7662090389818619
Switzerland                            Bern                               1.7135914194737725
Australia                              Canberra                           1.7087948745102193
South Africa                           Pretoria                           1.6312009312232212
Saint Lucia                            Castries                           1.4941558441558442
Kuwait                                 Kuwait                             1.4634381338742393
Belgium                                Bruxelles [Brussel]                1.3073444672331282
Brazil                                 Brasília                           1.1579625547423802
Netherlands Antilles                   Willemstad                         1.0806451612903225
Canada                                 Ottawa                             1.0764343275435837
Bhutan                                 Thimphu                            1.0357815442561205
Côte dIvoire                           Yamoussoukro                       0.8792100635736507
Guam                                   Agaña                              0.6779761904761905
China                                  Peking                             0.5848658143113659
Tanzania                               Dodoma                             0.5638929498463466
Pakistan                               Islamabad                          0.3351801793166031
Nigeria                                Abuja                              0.313974135920937
United States                          Washington                         0.2055127049077264
India                                  New Delhi                          0.029723615958771266
```

In [ ]:
pass1=
solution=$(cat ~/crypt/hive_example.crypt | openssl enc -md sha256 -aes-128-cbc -a -d -salt -pass pass:$pass1 2> /dev/null)
echo "$solution"; echo
for l in "$solution"; do eval "${l}"; done

## PIG

Pig is a scripting language for creating workflows based on MapReduce

Pig transforms the declarative nature of Hive into a procedural one, so that dataflow steps are more easily defined 

In this examples, we will use HCatalog to connect Pig to Hive databases. HCatalog is a table and storage management layer for Hadoop that enables users with different data processing tools — Pig, MapReduce — to more easily read and write data on the grid

In order run pig commands/scripts on hive tables, first ensure that hive metastore is running and listening:

In [ ]:
netstat -an | grep 9083

And start it if it is not:

In [ ]:
hive --service metastore &

We want pig to use HCatalog service to connect to hive. 

HCatalog is a table and storage management layer for Hadoop that enables users with different data processing tools — Pig, MapReduce — to more easily read and write data on the grid.

Let's check whether hcat is running:

In [ ]:
hcat -e "create table hcatalogtest(name string,place string,id int) row format delimited fields terminated by ':' stored as textfile" 2>&1 | \
grep -Pv "SLF4J|INFO|WARN"

In [ ]:
hcat -e "desc hcatalogtest" 2>&1 |  grep -Pv "SLF4J|INFO|WARN"

To enable pig with HCatalog:

```bash
pig -useHCatalog
```

We will write down the steps defining the workflow and the plan will be executed when we enter the DUMP command

As a simple example, let's load city table from world database in Hive

First we will list the steps and then run them as a single script in batch mode:

```Pig
city = LOAD 'world.city' USING org.apache.hive.hcatalog.pig.HCatLoader();
```

And let's select the cities where the countrycode is TUR

```Pig
cityturkey = filter city by countrycode == 'TUR';
```

And let's select the cities with a population larger than 1 million

```Pig
largecitytur = filter cityturkey by population > 1000000;
```

Now let's execute the plan:

```Pig
DUMP largecitytur;
```

Let's write down all steps in to a pig script file:

In [ ]:
cat > ~/script1.pig <<EOF
city = LOAD 'world.city' USING org.apache.hive.hcatalog.pig.HCatLoader();
cityturkey = filter city by countrycode == 'TUR';
largecitytur = filter cityturkey by population > 1000000;
DUMP largecitytur;
EOF

In [ ]:
cat ~/script1.pig

And execute the script:

In [ ]:
pig -useHCatalog ~/script1.pig 2>&1 | grep -Pv "INFO|WARN"

Now let's import other tables in the world database

```Pig
country = LOAD 'world.country' USING org.apache.hive.hcatalog.pig.HCatLoader();


lang = LOAD 'world.countrylanguage' USING org.apache.hive.hcatalog.pig.HCatLoader();
```

Now let's define our previous example in Hive as a Pig dataflow

First, filter the lang table for countries, official languages of which include English

```Pig
codeen = filter lang by language == 'English' and isofficial == true;
```

Then we join the filtered countrylanguage and country tables on the coeds

```Pig
joinen = JOIN country by code, codeen by countrycode;
```

And I select only the name field to be dumped

```Pig
names = foreach joinen generate name;
```

Now we can execute the plan to dump the names

```Pig
DUMP names;
```

In [ ]:
cat > ~/script2.pig <<EOF
country = LOAD 'world.country' USING org.apache.hive.hcatalog.pig.HCatLoader();
lang = LOAD 'world.countrylanguage' USING org.apache.hive.hcatalog.pig.HCatLoader();
codeen = filter lang by language == 'English' and isofficial == true;
joinen = JOIN country by code, codeen by countrycode;
names = foreach joinen generate name;
DUMP names;
EOF

In [ ]:
cat ~/script2.pig

In [ ]:
pig -useHCatalog ~/script2.pig 2>&1 | grep -Pv "INFO|WARN"

### PIG EXERCISE

Now as an exercise, we will compare the lifeexpentancy of the whole sample and the life expectancy of the countries with English as official language

To get a feel of calculating averages in Pig, below is the solution for the first part:

```Pig
-- filter for null values
countrynotnull = filter country by lifeexpectancy is not null;

-- get lifeexpectancy column

lifeall = foreach countrynotnull generate lifeexpectancy;

-- combine values into a single group 
lifeallg = group lifeall all;

-- calculate the average
avgall = foreach lifeallg generate AVG(lifeall);
-- execute
DUMP avgall;
```

The result is:
```Pig
(66.486036036036)
```

In [ ]:
cat > ~/script3.pig <<EOF
country = LOAD 'world.country' USING org.apache.hive.hcatalog.pig.HCatLoader(); -- load data
countrynotnull = filter country by lifeexpectancy is not null; -- filter for null values
lifeall = foreach countrynotnull generate lifeexpectancy; -- preprocess for grouping, lifeexpectancy versus countrynotnull
lifeallg = group lifeall all; -- create a single group of all to calculate the average lifeexpectany
avgall = foreach lifeallg generate AVG(lifeall); -- calculate the average life expectancy of all countries
DUMP avgall; -- execute and return
EOF

In [ ]:
cat ~/script3.pig

In [ ]:
pig -useHCatalog ~/script3.pig  2>&1 | grep -Pv "INFO|WARN"

Now play with code in previous example to get the lifeexpectancy values of English speaking countries (Note that we had extracted the names column. Just change the column)

And apply the steps above (you will have the life expectancies of anglophone countries instead of all countries, rest is the same)

Note that null elimination step is not necessary, the result is the same

The result should be:
```Pig
(71.5027027027027)
```

In [ ]:
cat > ~/script4.pig <<EOF
    country = LOAD 'world.country' USING org.apache.hive.hcatalog.pig.HCatLoader(); -- load country data
    countrynotnull = filter country by lifeexpectancy is not null; -- filter for null values
    lang = LOAD 'world.countrylanguage' USING org.apache.hive.hcatalog.pig.HCatLoader(); -- load language data
    codeen = filter lang by language == 'English' and isofficial == true; -- filter for English language
    joinen = JOIN countrynotnull by code, codeen by countrycode; -- join countries and English speaking ones
    lifeen = foreach joinen generate lifeexpectancy;  -- preprocess for grouping, lifeexpectancy versus countrynotnull

    lifeeng = group lifeen all; -- create a single group of all to calculate the average lifeexpectany
    avgen = foreach lifeeng generate AVG(lifeen); -- execute and return

    DUMP avgen;
EOF

In [ ]:
cat ~/script4.pig

In [ ]:
pig -useHCatalog ~/script4.pig  2>&1 | grep -Pv "INFO|WARN"

### REVIEW OF PIG COMMANDS

**LOAD:**

```Bash
    country = LOAD 'world.country' USING org.apache.hive.hcatalog.pig.HCatLoader();
```

**FILTER:**
```Bash
    countrynotnull = filter country by lifeexpectancy is not null;
```

**JOIN:**
```Bash
    joinen = JOIN countrynotnull by code, codeen by countrycode;
```

**FOREACH / GENERATE:** Calculate new fields from grouped or non-grouped objects. Also for selecting fields
```Bash
    lifeen = foreach joinen generate lifeexpectancy;
```

**GROUP:** Create a grouped object. Works like group by, summarize in dplyr. all means create a single group
```Bash
    lifeeng = group lifeen all;
        ot
    group object by field
```

**ORDER:**
```Bash
    ORDER variable/table by field (DESC);
```
**LIMIT:**
```Bash
    LIMIT variable/table N;
```

**DUMP:** Execute the whole script and return the results
```Bash
    DUMP avgen;
```

**Exercise 3:**

Write a pig workflow/script to return the countries capitals of which are also the cities with largest population

First let's check it with an SQL statement:

In [ ]:
psql -U postgres -d world <<EOF

with cmax as
 (
 select country.code, country.name, max(city.population) maxpop
 from country left join city on code=countrycode
 group by country.code, country.name
 )
 select country.code, country.name, city.name, city.population, cmax.maxpop
 from cmax left join country USING (code)
 left join city on capital=id
 where city.population=cmax.maxpop
 order by code;

EOF

You can use the below code for template, just fill in the xxx parts following the comments:

In [ ]:
cat > ~/script5.pig <<EOF
    country = LOAD 'world.country' USING org.apache.hive.hcatalog.pig.HCatLoader(); -- load country data
    country2 = foreach country generate code, name, capital; -- select necessary fields
    city = LOAD 'world.city' USING org.apache.hive.hcatalog.pig.HCatLoader(); -- load city data
    city2 = foreach city generate id, name as cityname, countrycode, population; -- select necessary fields
    countrycity = JOIN xxx by xxx, xxx by xxx; -- join country to city data
    citiesbyc = group xxx by xxx; -- create a group of the countrycity for each country

    -- for each group in the citiesbyc, return the country code and maximum population
    maxcity = foreach xxx generate group as code, MAX(countrycity.xxx) as maxpop;
    countrycap = JOIN xxx by xxx, xxx by xxx; -- join the capital city information to countries using country2 and city2
    countrycapmax = JOIN xxx by code, maxcity by code; -- join country information with capital cities and country information with maximum population
    countryfiltered = FILTER xxx by xxx == xxx; -- filter where capital population is equal to maximum popluation

    DUMP countryfiltered;
EOF

And try whether it runs:

Note that, at any step, you can comment out the lines below and execute the script to DUMP the last calculated object for debugging purposes 

In [ ]:
cat ~/script5.pig

pig -useHCatalog ~/script5.pig

The output should be as such:


```Bash
    DUMP countryfiltered;
(ABW,Aruba,129,129,Oranjestad,ABW,29034,ABW,29034)
(AFG,Afghanistan,1,1,Kabul,AFG,1780000,AFG,1780000)
(AGO,Angola,56,56,Luanda,AGO,2022000,AGO,2022000)
(ALB,Albania,34,34,Tirana,ALB,270000,ALB,270000)
(AND,Andorra,55,55,Andorra la Vella,AND,21189,AND,21189)
(ANT,Netherlands Antilles,33,33,Willemstad,ANT,2345,ANT,2345)
(ARG,Argentina,69,69,Buenos Aires,ARG,2982146,ARG,2982146)
(ARM,Armenia,126,126,Yerevan,ARM,1248700,ARM,1248700)
(ATG,Antigua and Barbuda,63,63,Saint John´s,ATG,24000,ATG,24000)
(AUT,Austria,1523,1523,Wien,AUT,1608144,AUT,1608144)
(AZE,Azerbaijan,144,144,Baku,AZE,1787800,AZE,1787800)
(BDI,Burundi,552,552,Bujumbura,BDI,300000,BDI,300000)
(BFA,Burkina Faso,549,549,Ouagadougou,BFA,824000,BFA,824000)
(BGD,Bangladesh,150,150,Dhaka,BGD,3612850,BGD,3612850)
(BGR,Bulgaria,539,539,Sofija,BGR,1122302,BGR,1122302)
(BHR,Bahrain,149,149,al-Manama,BHR,148000,BHR,148000)
(BHS,Bahamas,148,148,Nassau,BHS,172000,BHS,172000)
(BIH,Bosnia and Herzegovina,201,201,Sarajevo,BIH,360000,BIH,360000)
(BLR,Belarus,3520,3520,Minsk,BLR,1674000,BLR,1674000)
(BRB,Barbados,174,174,Bridgetown,BRB,6070,BRB,6070)
(BRN,Brunei,538,538,Bandar Seri Begawan,BRN,21484,BRN,21484)
(BTN,Bhutan,192,192,Thimphu,BTN,22000,BTN,22000)
(BWA,Botswana,204,204,Gaborone,BWA,213017,BWA,213017)
(CAF,Central African Republic,1889,1889,Bangui,CAF,524000,CAF,524000)
(CHL,Chile,554,554,Santiago de Chile,CHL,4703954,CHL,4703954)
(COD,Congo, The Democratic Republic of the,2298,2298,Kinshasa,COD,5064000,COD,5064000)
(COG,Congo,2296,2296,Brazzaville,COG,950000,COG,950000)
(COK,Cook Islands,583,583,Avarua,COK,11900,COK,11900)
(COL,Colombia,2257,2257,Santafé de Bogotá,COL,6260862,COL,6260862)
(COM,Comoros,2295,2295,Moroni,COM,36000,COM,36000)
(CPV,Cape Verde,1859,1859,Praia,CPV,94800,CPV,94800)
(CRI,Costa Rica,584,584,San José,CRI,339131,CRI,339131)
(CUB,Cuba,2413,2413,La Habana,CUB,2256000,CUB,2256000)
(CXR,Christmas Island,1791,1791,Flying Fish Cove,CXR,700,CXR,700)
(CYM,Cayman Islands,553,553,George Town,CYM,19600,CYM,19600)
(CYP,Cyprus,2430,2430,Nicosia,CYP,195000,CYP,195000)
(CZE,Czech Republic,3339,3339,Praha,CZE,1181126,CZE,1181126)
(DEU,Germany,3068,3068,Berlin,DEU,3386667,DEU,3386667)
(DJI,Djibouti,585,585,Djibouti,DJI,383000,DJI,383000)
(DMA,Dominica,586,586,Roseau,DMA,16243,DMA,16243)
(DNK,Denmark,3315,3315,København,DNK,495699,DNK,495699)
(DOM,Dominican Republic,587,587,Santo Domingo de Guzmán,DOM,1609966,DOM,1609966)
(DZA,Algeria,35,35,Alger,DZA,2168000,DZA,2168000)
(EGY,Egypt,608,608,Cairo,EGY,6789479,EGY,6789479)
(ERI,Eritrea,652,652,Asmara,ERI,431000,ERI,431000)
(ESH,Western Sahara,2453,2453,El-Aaiún,ESH,169000,ESH,169000)
(ESP,Spain,653,653,Madrid,ESP,2879052,ESP,2879052)
(EST,Estonia,3791,3791,Tallinn,EST,403981,EST,403981)
(ETH,Ethiopia,756,756,Addis Abeba,ETH,2495000,ETH,2495000)
(FIN,Finland,3236,3236,Helsinki [Helsingfors],FIN,555474,FIN,555474)
(FJI,Fiji Islands,764,764,Suva,FJI,77366,FJI,77366)
(FLK,Falkland Islands,763,763,Stanley,FLK,1636,FLK,1636)
(FRA,France,2974,2974,Paris,FRA,2125246,FRA,2125246)
(FRO,Faroe Islands,901,901,Tórshavn,FRO,14542,FRO,14542)
(GAB,Gabon,902,902,Libreville,GAB,419000,GAB,419000)
(GBR,United Kingdom,456,456,London,GBR,7285000,GBR,7285000)
(GEO,Georgia,905,905,Tbilisi,GEO,1235200,GEO,1235200)
(GHA,Ghana,910,910,Accra,GHA,1070000,GHA,1070000)
(GIB,Gibraltar,915,915,Gibraltar,GIB,27025,GIB,27025)
(GIN,Guinea,926,926,Conakry,GIN,1090610,GIN,1090610)
(GNB,Guinea-Bissau,927,927,Bissau,GNB,241000,GNB,241000)
(GNQ,Equatorial Guinea,2972,2972,Malabo,GNQ,40000,GNQ,40000)
(GRC,Greece,2401,2401,Athenai,GRC,772072,GRC,772072)
(GRD,Grenada,916,916,Saint George´s,GRD,4621,GRD,4621)
(GRL,Greenland,917,917,Nuuk,GRL,13445,GRL,13445)
(GTM,Guatemala,922,922,Ciudad de Guatemala,GTM,823301,GTM,823301)
(GUF,French Guiana,3014,3014,Cayenne,GUF,50699,GUF,50699)
(GUY,Guyana,928,928,Georgetown,GUY,254000,GUY,254000)
(HND,Honduras,933,933,Tegucigalpa,HND,813900,HND,813900)
(HRV,Croatia,2409,2409,Zagreb,HRV,706770,HRV,706770)
(HTI,Haiti,929,929,Port-au-Prince,HTI,884472,HTI,884472)
(HUN,Hungary,3483,3483,Budapest,HUN,1811552,HUN,1811552)
(IDN,Indonesia,939,939,Jakarta,IDN,9604900,IDN,9604900)
(IRL,Ireland,1447,1447,Dublin,IRL,481854,IRL,481854)
(IRN,Iran,1380,1380,Teheran,IRN,6758845,IRN,6758845)
(IRQ,Iraq,1365,1365,Baghdad,IRQ,4336000,IRQ,4336000)
(ISL,Iceland,1449,1449,Reykjavík,ISL,109184,ISL,109184)
(ISR,Israel,1450,1450,Jerusalem,ISR,633700,ISR,633700)
(ITA,Italy,1464,1464,Roma,ITA,2643581,ITA,2643581)
(JOR,Jordan,1786,1786,Amman,JOR,1000000,JOR,1000000)
(JPN,Japan,1532,1532,Tokyo,JPN,7980230,JPN,7980230)
(KEN,Kenya,1881,1881,Nairobi,KEN,2290000,KEN,2290000)
(KGZ,Kyrgyzstan,2253,2253,Bishkek,KGZ,589400,KGZ,589400)
(KHM,Cambodia,1800,1800,Phnom Penh,KHM,570155,KHM,570155)
(KNA,Saint Kitts and Nevis,3064,3064,Basseterre,KNA,11600,KNA,11600)
(KOR,South Korea,2331,2331,Seoul,KOR,9981619,KOR,9981619)
(LAO,Laos,2432,2432,Vientiane,LAO,531800,LAO,531800)
(LBN,Lebanon,2438,2438,Beirut,LBN,1100000,LBN,1100000)
(LBR,Liberia,2440,2440,Monrovia,LBR,850000,LBR,850000)
(LBY,Libyan Arab Jamahiriya,2441,2441,Tripoli,LBY,1682000,LBY,1682000)
(LCA,Saint Lucia,3065,3065,Castries,LCA,2301,LCA,2301)
(LKA,Sri Lanka,3217,3217,Colombo,LKA,645000,LKA,645000)
(LSO,Lesotho,2437,2437,Maseru,LSO,297000,LSO,297000)
(LTU,Lithuania,2447,2447,Vilnius,LTU,577969,LTU,577969)
(LUX,Luxembourg,2452,2452,Luxembourg [Luxemburg/Lëtzebuerg],LUX,80700,LUX,80700)
(LVA,Latvia,2434,2434,Riga,LVA,764328,LVA,764328)
(MAC,Macao,2454,2454,Macao,MAC,437500,MAC,437500)
(MDA,Moldova,2690,2690,Chisinau,MDA,719900,MDA,719900)
(MDG,Madagascar,2455,2455,Antananarivo,MDG,675669,MDG,675669)
(MDV,Maldives,2463,2463,Male,MDV,71000,MDV,71000)
(MEX,Mexico,2515,2515,Ciudad de México,MEX,8591309,MEX,8591309)
(MHL,Marshall Islands,2507,2507,Dalap-Uliga-Darrit,MHL,28000,MHL,28000)
(MKD,Macedonia,2460,2460,Skopje,MKD,444299,MKD,444299)
(MLI,Mali,2482,2482,Bamako,MLI,809552,MLI,809552)
(MMR,Myanmar,2710,2710,Rangoon (Yangon),MMR,3361700,MMR,3361700)
(MNG,Mongolia,2696,2696,Ulan Bator,MNG,773700,MNG,773700)
(MNP,Northern Mariana Islands,2913,2913,Garapan,MNP,9200,MNP,9200)
(MOZ,Mozambique,2698,2698,Maputo,MOZ,1018938,MOZ,1018938)
(MRT,Mauritania,2509,2509,Nouakchott,MRT,667300,MRT,667300)
(MSR,Montserrat,2697,2697,Plymouth,MSR,2000,MSR,2000)
(MTQ,Martinique,2508,2508,Fort-de-France,MTQ,94050,MTQ,94050)
(MUS,Mauritius,2511,2511,Port-Louis,MUS,138200,MUS,138200)
(MYS,Malaysia,2464,2464,Kuala Lumpur,MYS,1297526,MYS,1297526)
(MYT,Mayotte,2514,2514,Mamoutzou,MYT,12000,MYT,12000)
(NAM,Namibia,2726,2726,Windhoek,NAM,169000,NAM,169000)
(NCL,New Caledonia,3493,3493,Nouméa,NCL,76293,NCL,76293)
(NER,Niger,2738,2738,Niamey,NER,420000,NER,420000)
(NFK,Norfolk Island,2806,2806,Kingston,NFK,800,NFK,800)
(NIC,Nicaragua,2734,2734,Managua,NIC,959000,NIC,959000)
(NIU,Niue,2805,2805,Alofi,NIU,682,NIU,682)
(NLD,Netherlands,5,5,Amsterdam,NLD,731200,NLD,731200)
(NOR,Norway,2807,2807,Oslo,NOR,508726,NOR,508726)
(NPL,Nepal,2729,2729,Kathmandu,NPL,591835,NPL,591835)
(PAN,Panama,2882,2882,Ciudad de Panamá,PAN,471373,PAN,471373)
(PCN,Pitcairn,2912,2912,Adamstown,PCN,42,PCN,42)
(PER,Peru,2890,2890,Lima,PER,6464693,PER,6464693)
(PLW,Palau,2881,2881,Koror,PLW,12000,PLW,12000)
(PNG,Papua New Guinea,2884,2884,Port Moresby,PNG,247000,PNG,247000)
(POL,Poland,2928,2928,Warszawa,POL,1615369,POL,1615369)
(PRI,Puerto Rico,2919,2919,San Juan,PRI,434374,PRI,434374)
(PRK,North Korea,2318,2318,Pyongyang,PRK,2484000,PRK,2484000)
(PRT,Portugal,2914,2914,Lisboa,PRT,563210,PRT,563210)
(PRY,Paraguay,2885,2885,Asunción,PRY,557776,PRY,557776)
(PSE,Palestine,4074,4074,Gaza,PSE,353632,PSE,353632)
(QAT,Qatar,2973,2973,Doha,QAT,355000,QAT,355000)
(REU,Réunion,3017,3017,Saint-Denis,REU,131480,REU,131480)
(ROM,Romania,3018,3018,Bucuresti,ROM,2016131,ROM,2016131)
(RUS,Russian Federation,3580,3580,Moscow,RUS,8389200,RUS,8389200)
(RWA,Rwanda,3047,3047,Kigali,RWA,286000,RWA,286000)
(SAU,Saudi Arabia,3173,3173,Riyadh,SAU,3324000,SAU,3324000)
(SGP,Singapore,3208,3208,Singapore,SGP,4017733,SGP,4017733)
(SHN,Saint Helena,3063,3063,Jamestown,SHN,1500,SHN,1500)
(SJM,Svalbard and Jan Mayen,938,938,Longyearbyen,SJM,1438,SJM,1438)
(SLB,Solomon Islands,3161,3161,Honiara,SLB,50100,SLB,50100)
(SLE,Sierra Leone,3207,3207,Freetown,SLE,850000,SLE,850000)
(SLV,El Salvador,645,645,San Salvador,SLV,415346,SLV,415346)
(SOM,Somalia,3214,3214,Mogadishu,SOM,997000,SOM,997000)
(SPM,Saint Pierre and Miquelon,3067,3067,Saint-Pierre,SPM,5808,SPM,5808)
(STP,Sao Tome and Principe,3172,3172,São Tomé,STP,49541,STP,49541)
(SUR,Suriname,3243,3243,Paramaribo,SUR,112000,SUR,112000)
(SVK,Slovakia,3209,3209,Bratislava,SVK,448292,SVK,448292)
(SVN,Slovenia,3212,3212,Ljubljana,SVN,270986,SVN,270986)
(SWE,Sweden,3048,3048,Stockholm,SWE,750348,SWE,750348)
(SWZ,Swaziland,3244,3244,Mbabane,SWZ,61000,SWZ,61000)
(SYC,Seychelles,3206,3206,Victoria,SYC,41000,SYC,41000)
(SYR,Syria,3250,3250,Damascus,SYR,1347000,SYR,1347000)
(TCA,Turks and Caicos Islands,3423,3423,Cockburn Town,TCA,4800,TCA,4800)
(TCD,Chad,3337,3337,N´Djaména,TCD,530965,TCD,530965)
(TGO,Togo,3332,3332,Lomé,TGO,375000,TGO,375000)
(THA,Thailand,3320,3320,Bangkok,THA,6320174,THA,6320174)
(TJK,Tajikistan,3261,3261,Dushanbe,TJK,524000,TJK,524000)
(TKL,Tokelau,3333,3333,Fakaofo,TKL,300,TKL,300)
(TKM,Turkmenistan,3419,3419,Ashgabat,TKM,540600,TKM,540600)
(TMP,East Timor,1522,1522,Dili,TMP,47900,TMP,47900)
(TON,Tonga,3334,3334,Nuku´alofa,TON,22400,TON,22400)
(TUN,Tunisia,3349,3349,Tunis,TUN,690600,TUN,690600)
(TUV,Tuvalu,3424,3424,Funafuti,TUV,4600,TUV,4600)
(TWN,Taiwan,3263,3263,Taipei,TWN,2641312,TWN,2641312)
(UGA,Uganda,3425,3425,Kampala,UGA,890800,UGA,890800)
(UKR,Ukraine,3426,3426,Kyiv,UKR,2624000,UKR,2624000)
(URY,Uruguay,3492,3492,Montevideo,URY,1236000,URY,1236000)
(UZB,Uzbekistan,3503,3503,Toskent,UZB,2117500,UZB,2117500)
(VAT,Holy See (Vatican City State),3538,3538,Città del Vaticano,VAT,455,VAT,455)
(VCT,Saint Vincent and the Grenadines,3066,3066,Kingstown,VCT,17100,VCT,17100)
(VEN,Venezuela,3539,3539,Caracas,VEN,1975294,VEN,1975294)
(VGB,Virgin Islands, British,537,537,Road Town,VGB,8000,VGB,8000)
(VIR,Virgin Islands, U.S.,4067,4067,Charlotte Amalie,VIR,13000,VIR,13000)
(VUT,Vanuatu,3537,3537,Port-Vila,VUT,33700,VUT,33700)
(WLF,Wallis and Futuna,3536,3536,Mata-Utu,WLF,1137,WLF,1137)
(WSM,Samoa,3169,3169,Apia,WSM,35900,WSM,35900)
(YEM,Yemen,1780,1780,Sanaa,YEM,503600,YEM,503600)
(YUG,Yugoslavia,1792,1792,Beograd,YUG,1204000,YUG,1204000)
(ZMB,Zambia,3162,3162,Lusaka,ZMB,1317000,ZMB,1317000)
(ZWE,Zimbabwe,4068,4068,Harare,ZWE,1410000,ZWE,1410000)
```

In [ ]:
pass1=
solution=$(cat ~/crypt/pig_example.crypt | openssl enc -md sha256 -aes-128-cbc -a -d -salt -pass pass:$pass1 2> /dev/null)
echo "$solution"; echo
for l in "$solution"; do eval "${l}"; done